In [1]:
!pip install sentence-transformers bertopic

  Using cached bertopic-0.17.0-py3-none-any.whl.metadata (23 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manyl

In [2]:
import pandas as pd
import numpy as np
from umap import UMAP
from hdbscan import HDBSCAN
from tqdm.notebook import tqdm
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy.spatial.distance import cosine
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer

In [3]:
# Download L'éducation sentimentale and Vingt mille lieux sous les mers on your drive
!gdown 1A6P3_YVUo-DYDbpAn_RDA4pRFydI-4ju
!gdown 1ZtOZ7UewZaR-XUSH01W2CDfrc6v8EU-s

Downloading...
From: https://drive.google.com/uc?id=1A6P3_YVUo-DYDbpAn_RDA4pRFydI-4ju
To: /content/1870_Verne-Jules_Vingt-mille-lieues-sous-les-mers.txt
100% 912k/912k [00:00<00:00, 9.59MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZtOZ7UewZaR-XUSH01W2CDfrc6v8EU-s
To: /content/1869_Flaubert-Gustave_L-Education-sentimentale.txt
100% 1.00M/1.00M [00:00<00:00, 10.4MB/s]


In [4]:
!gdown 1x_8fZFqOrJA3hj_HPJ7_ctS7ttZWxlxA # download list of french stopwords

Downloading...
From: https://drive.google.com/uc?id=1x_8fZFqOrJA3hj_HPJ7_ctS7ttZWxlxA
To: /content/stopwords.txt
100% 4.59k/4.59k [00:00<00:00, 12.9MB/s]


In [5]:
def read_and_chunk_text(path_to_txt, chunk_size=256, overlap=64):
    with open(path_to_txt, 'r', encoding='utf-8') as f:
        text = f.read()

    # Tokenize the text by splitting on whitespace
    tokens = text.split()

    # Create rolling chunks
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = tokens[i:i + chunk_size]
        if len(chunk) == chunk_size:
            chunks.append(chunk)

    return chunks

In [6]:
chunks_flaubert = read_and_chunk_text("1869_Flaubert-Gustave_L-Education-sentimentale.txt")
chunks_verne = read_and_chunk_text("1870_Verne-Jules_Vingt-mille-lieues-sous-les-mers.txt")

In [7]:
stops = open("stopwords.txt", encoding="utf-8").read().split("\n")

In [8]:
# Load the model
sentence_model = SentenceTransformer('crazyjeannot/fr_literary_bge_base')
#sentence_model = SentenceTransformer('Alibaba-NLP/gte-base-en-v1.5', trust_remote_code=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.27k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [9]:
# Your documents as a list of strings
documents_as_strings = [" ".join(tokens) for tokens in chunks_flaubert]

In [10]:
len(documents_as_strings) # L'éducation sentimentale has 811 chunks of 256 tokens

811

In [11]:
documents_as_strings[0] # First chunk

'Le 15 septembre 1840, vers six heures du matin, la Ville-de-Montereau, près de partir, fumait à gros tourbillons devant le quai Saint-Bernard.Des gens arrivaient hors d’haleine ; des barriques, des câbles, des corbeilles de linge gênaient la circulation ; les matelots ne répondaient à personne ; on se heurtait ; les colis montaient entre les deux tambours, et le tapage s’absorbait dans le bruissement de la vapeur, qui, s’échappant par des plaques de tôle, enveloppait tout d’une nuée blanchâtre, tandis que la cloche, à l’avant, tintait sans discontinuer.Enfin le navire partit ; et les deux berges, peuplées de magasins, de chantiers et d’usines, filèrent comme deux larges rubans que l’on déroule.Un jeune homme de dix-huit ans, à longs cheveux et qui tenait un album sous son bras, restait auprès du gouvernail, immobile. À travers le brouillard, il contemplait des clochers, des édifices dont il ne savait pas les noms ; puis il embrassa, dans un dernier coup d’œil, l’île Saint-Louis, la Ci

In [12]:
!pip install spacy
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 84.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [13]:
import spacy
nlp = spacy.load("fr_core_news_sm")

In [16]:
embeddings = sentence_model.encode(documents_as_strings, show_progress_bar=True)

Batches:   0%|          | 0/26 [00:00<?, ?it/s]

In [17]:
vectorizer_model = CountVectorizer(stop_words=stops)

In [18]:
# Instantiate UMAP model
umap_model = UMAP(n_neighbors=10, n_components=10, metric='cosine')

# Instantiate HDBSCAN model
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean',
                        cluster_selection_method='eom', cluster_selection_epsilon=0.1)

In [19]:
# Instantiate BERTopic model
topic_model = BERTopic(
    umap_model=umap_model,  # Pass the instantiated UMAP model
    hdbscan_model=hdbscan_model,  # Pass the instantiated HDBSCAN model
    min_topic_size=10,  # Minimum size of clusters
    top_n_words=10,
    nr_topics=50,
    vectorizer_model=vectorizer_model,
    verbose=True,
    #calculate_probabilities=False,
    #low_memory=True
)

In [20]:
# Fit the BERTopic model to your data and generate topics
topics, probs = topic_model.fit_transform(documents_as_strings, embeddings)

2025-04-10 09:01:11,349 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-10 09:01:21,120 - BERTopic - Dimensionality - Completed ✓
2025-04-10 09:01:21,121 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-10 09:01:21,152 - BERTopic - Cluster - Completed ✓
2025-04-10 09:01:21,153 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-04-10 09:01:21,368 - BERTopic - Representation - Completed ✓
2025-04-10 09:01:21,369 - BERTopic - Topic reduction - Reducing number of topics
2025-04-10 09:01:21,371 - BERTopic - Topic reduction - Number of topics (50) is equal or higher than the clustered topics(15).
2025-04-10 09:01:21,371 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-10 09:01:21,570 - BERTopic - Representation - Completed ✓


In [21]:
# Display topics
print(topic_model.get_topic_info())

    Topic  Count                  Name  \
0      -1    381        -1_de_la_et_le   
1       0     99        0_de_la_le_les   
2       1     38  1_de_dambreuse_et_le   
3       2     66       2_de_la_et_elle   
4       3     17      3_de_sncal_il_le   
5       4     14       4_des_les_de_le   
6       5     12         5_il_le_de_la   
7       6     28         6_de_et_la_il   
8       7     14       7_de_le_les_des   
9       8     17        8_des_de_et_un   
10      9     16       9_des_les_de_et   
11     10     14   10_elle_de_et_mille   
12     11     41    11_elle_vous_de_la   
13     12     14        12_le_de_et_il   
14     13     40        13_de_il_le_et   

                                       Representation  \
0         [de, la, et, le, il, les, des, un, une, se]   
1          [de, la, le, les, et, des, du, en, il, un]   
2   [de, dambreuse, et, le, la, il, les, une, un, ...   
3   [de, la, et, elle, le, rosanette, un, les, il,...   
4       [de, sncal, il, le, et, la, les, s

In [22]:
print(topic_model.get_topic(0))  # Replace 0 with the topic ID

[('de', np.float64(0.048045057214692155)), ('la', np.float64(0.04639719772832143)), ('le', np.float64(0.03802064015006242)), ('les', np.float64(0.03387087097176751)), ('et', np.float64(0.03032721280160526)), ('des', np.float64(0.02651569320890551)), ('du', np.float64(0.02634299402355726)), ('en', np.float64(0.026177862615071337)), ('il', np.float64(0.02412347836210586)), ('un', np.float64(0.021796647157073017))]


In [23]:
# Topic visualization
topic_model.visualize_topics()

# Intertopic distance map
topic_model.visualize_barchart(top_n_topics=10)  # Bar chart for top 10 topics

In [16]:
def remove_proper_names(text):
    """Remove tokens that are proper nouns or PERSON entities, preserving accented characters."""
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        # Remove proper nouns or PERSON entities
        if token.pos_ == "PROPN" or token.ent_type_ == "PERSON":
            continue
        filtered_tokens.append(token.text)
    return " ".join(filtered_tokens)


In [15]:
len(documents_as_strings)

811

In [17]:
clean_documents = [remove_proper_names(doc) for doc in documents_as_strings]
len(clean_documents)

811

In [29]:
clean_documents[450]

'place d’ attaché au Parquet devait lui fournir des renseignements . Il confessa en pas avoir . Du reste , il connaissait fort peu le personnage , l’ ayant vu deux ou trois fois seulement , il le tenait en définitive pour un assez mauvais drôle . , indigné , s’ écria : – Pas du tout ! c’ est un très honnête garçon ! – Cependant , monsieur , dit un propriétaire , on n’ est pas honnête quand on conspire ! La plupart des hommes qui étaient là avaient servi , au moins , quatre gouvernements ; et ils auraient vendu la ou le genre humain , pour garantir leur fortune , s’ épargner un malaise , un embarras , ou même par simple bassesse , adoration instinctive de la force . Tous déclarèrent les crimes politiques inexcusables . Il fallait plutôt pardonner à ceux qui provenaient du besoin ! Et on ne manqua pas de mettre en avant l’ éternel exemple du père de famille , volant l’ éternel morceau de pain chez l’ éternel boulanger . Un administrateur s’ écria même : – , monsieur , si j’ apprenais que

In [30]:
import numpy as np
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from sklearn.feature_extraction.text import TfidfVectorizer
import umap
import hdbscan

# On suppose que `documents_as_strings` est votre liste de documents textuels,
# et que `stops` est votre liste prédéfinie de mots vides (stop words).

# On nettoie les documents en supprimant les noms propres
#clean_documents = [remove_proper_names(doc) for doc in documents_as_strings]

# 1. Calcul des embeddings des phrases
print("Chargement du modèle SentenceTransformer ...")
#sentence_model = SentenceTransformer("Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)
#sentence_model = SentenceTransformer('crazyjeannot/fr_literary_bge_base')

#embeddings = sentence_model.encode(clean_documents, show_progress_bar=True)
print("Embeddings calculés. Dimensions :", np.array(embeddings).shape)

# 2. Définition du TfidfVectorizer avec des hyperparamètres supplémentaires
#    pour supprimer les tokens extrêmement courants ou rares.
vectorizer_model = TfidfVectorizer(
    stop_words=stops,
    ngram_range=(1, 2),
    min_df=2,             # Conserver uniquement les mots apparaissant dans au moins 3 documents
    max_df=500,            # Ignorer les mots apparaissant dans plus de 50 documents
    max_features=1000,   # Limiter le nombre de caractéristiques aux 10 000 tokens les plus fréquents
    strip_accents=None,   # Ne pas supprimer les accents
    token_pattern=r"(?u)\b\w+\b"  # Ce motif détecte les caractères accentués en mode Unicode
)

# 3. Configuration de UMAP pour la réduction de dimension avec des hyperparamètres ajustés
umap_model = umap.UMAP(
    n_neighbors=5,    # Utiliser un nombre de voisins plus faible pour capturer des structures locales
    n_components=5,   # Réduire à 5 dimensions
    metric='cosine',
    random_state=42
)

# 4. Configuration de HDBSCAN pour le clustering sur les données réduites par UMAP
hdbscan_model = hdbscan.HDBSCAN(
    min_cluster_size=5,                  # Taille minimale d'un cluster pour être considérée
    metric='euclidean',
    cluster_selection_method='eom',
    cluster_selection_epsilon=0.1        # Vous pouvez expérimenter cette valeur
)

# 5. Instanciation du modèle BERTopic avec fusion automatique des topics
topic_model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    min_topic_size=5,   # Taille minimale d'un topic pour être conservé
    top_n_words=10,
    nr_topics="auto",   # Laisser BERTopic décider automatiquement de la fusion des topics
    verbose=True
)

# 6. Ajustement du modèle BERTopic sur les documents nettoyés et leurs embeddings
topics, probs = topic_model.fit_transform(clean_documents, embeddings)

# 7. Affichage des informations sur les topics pour inspection
print(topic_model.get_topic_info())


2025-04-10 13:56:13,062 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


Chargement du modèle SentenceTransformer ...
Embeddings calculés. Dimensions : (811, 768)


2025-04-10 13:56:16,545 - BERTopic - Dimensionality - Completed ✓
2025-04-10 13:56:16,546 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-10 13:56:16,575 - BERTopic - Cluster - Completed ✓
2025-04-10 13:56:16,576 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-04-10 13:56:17,035 - BERTopic - Representation - Completed ✓
2025-04-10 13:56:17,036 - BERTopic - Topic reduction - Reducing number of topics
2025-04-10 13:56:17,045 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-10 13:56:17,502 - BERTopic - Representation - Completed ✓
2025-04-10 13:56:17,504 - BERTopic - Topic reduction - Reduced number of topics from 2 to 2


   Topic  Count                       Name  \
0      0    801    0_tait_mme_arnoux_faire   
1      1     10  1_rosanette_mme_tait_bras   

                                      Representation  \
0  [tait, mme, arnoux, faire, taient, aprs, homme...   
1  [rosanette, mme, tait, bras, yeux, arnoux, jam...   

                                 Representative_Docs  
0  [, un peu plus loin , écrivait . Il commença p...  
1  [et arrivèrent sur le pont Royal . Il faisait ...  


In [ ]:
# Topic visualization
topic_model.visualize_topics()

# Intertopic distance map
topic_model.visualize_barchart(top_n_topics=20)  # Bar chart for top 10 topics